# Financial Market Analytics


> Google Cloud,

> Yahoo Finance,

>  S&P 500 Indices,

> Open Source, Business Analytics

> Financial Analytics and Entrepreneurial drive.

> Note: Financial markets are subject to highly volatile and non-stationary. AI and Data exploration purpose.





# Healthcare Markets - Financial Analysis

In [ ]:
!pip install pandas_ta ta
import yfinance as yf
import pandas as pd
import ta
import numpy as np
from google.colab import drive
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module='ta.trend')

# Mount Google Drive to access files
drive.mount('/content/drive', force_remount=True)

# File paths
drive_path = '/content/drive/My Drive/Colab Notebooks'
input_file_path = f'{drive_path}/Stock_List.xlsx'
output_file_path = f'{drive_path}/BB_Results_v5A_US1000_12cons_2RSI_Backtesting_v1.xlsx'
sheet_name = 'US_1000'

# Function to calculate indicators for a given ticker
def calculate_indicators(ticker):
    data = yf.download(ticker, period='1y', interval='1d')
    if data.empty or len(data) < 200:  # Ensure there is enough data for long-term indicators
        print(f"Insufficient data for {ticker}")
        return pd.DataFrame()  # Skip this stock

    # The rest of your indicator calculations


    # Calculate scores for today and yesterday
    def get_score(data, day_offset=0):
        score = 0
        results = {}

        # MACD
        macd = ta.trend.MACD(data['Close'])
        macd_score = int(macd.macd_diff().iloc[-1 - day_offset] > 0)
        results['MACD'] = macd_score
        score += macd_score

        # RSI with additional condition (RSI yesterday > RSI day before yesterday)
        rsi = ta.momentum.RSIIndicator(data['Close']).rsi()
        rsi_score = int(30 < rsi.iloc[-1 - day_offset] < 60 and rsi.iloc[-1 - day_offset] > rsi.iloc[-2 - day_offset])
        results['RSI'] = rsi_score
        score += rsi_score

        # Golden Cross
        sma50 = ta.trend.SMAIndicator(data['Close'], 50).sma_indicator()
        sma200 = ta.trend.SMAIndicator(data['Close'], 200).sma_indicator()
        ma_score = int(sma50.iloc[-1 - day_offset] > sma200.iloc[-1 - day_offset])
        results['Golden Cross'] = ma_score
        score += ma_score

        # # Bollinger Bands
        # bb = ta.volatility.BollingerBands(data['Close'])
        # bb_score = int(data['Close'].iloc[-1 - day_offset] > bb.bollinger_hband().iloc[-1 - day_offset])
        # results['Bollinger Bands'] = bb_score
        # score += bb_score

        # Bollinger Bands, # Using the middle band (20-day SMA) instead of the upper band

        bb = ta.volatility.BollingerBands(data['Close'])
        middle_band = bb.bollinger_mavg()  # This represents the 20-day SMA
             # Scoring logic: 1 point if the price is above the middle Bollinger Band
        bb_score = int(data['Close'].iloc[-1 - day_offset] > middle_band.iloc[-1 - day_offset])
            # Add this to the results and overall score
        results['Bollinger Bands'] = bb_score
        score += bb_score

        # Volume
        avg_vol = data['Volume'].rolling(window=20).mean()
        vol_score = int(data['Volume'].iloc[-1 - day_offset] > avg_vol.iloc[-1 - day_offset])
        results['Volume'] = vol_score
        score += vol_score

        # OBV
        obv = ta.volume.OnBalanceVolumeIndicator(data['Close'], data['Volume']).on_balance_volume()
        obv_score = int(obv.diff().iloc[-1 - day_offset] > 0)
        results['OBV'] = obv_score
        score += obv_score

        # Accumulation/Distribution Line
        ad = ta.volume.AccDistIndexIndicator(data['High'], data['Low'], data['Close'], data['Volume']).acc_dist_index()
        ad_score = int(ad.diff().iloc[-1 - day_offset] > 0)
        results['Acc/Dist Line'] = ad_score
        score += ad_score

        # Chaikin Money Flow
        cmf = ta.volume.ChaikinMoneyFlowIndicator(data['High'], data['Low'], data['Close'], data['Volume']).chaikin_money_flow()
        cmf_score = int(cmf.iloc[-1 - day_offset] > 0)
        results['CMF'] = cmf_score
        score += cmf_score

        # Parabolic SAR
        psar = ta.trend.PSARIndicator(data['High'], data['Low'], data['Close']).psar()
        psar_score = int(data['Close'].iloc[-1 - day_offset] > psar.iloc[-1 - day_offset])
        results['Parabolic SAR'] = psar_score
        score += psar_score

        # Stochastic Oscillator
        stoch = ta.momentum.StochasticOscillator(data['High'], data['Low'], data['Close'])
        stoch_score = int(stoch.stoch_signal().iloc[-1 - day_offset] > stoch.stoch().iloc[-1 - day_offset] and stoch.stoch().iloc[-1 - day_offset] < 20)
        results['Stochastic'] = stoch_score
        score += stoch_score

        # Fibonacci Retracement
        max_price = data['Close'].max()
        min_price = data['Close'].min()
        fib_50 = min_price + (max_price - min_price) * 0.5
        fib_score = int(data['Close'].iloc[-1 - day_offset] > fib_50)
        results['Fib Retracement'] = fib_score
        score += fib_score

        # ADX
        adx = ta.trend.ADXIndicator(data['High'], data['Low'], data['Close']).adx()
        plus_di = ta.trend.ADXIndicator(data['High'], data['Low'], data['Close']).adx_pos()
        minus_di = ta.trend.ADXIndicator(data['High'], data['Low'], data['Close']).adx_neg()
        adx_score = int(adx.iloc[-1 - day_offset] > 20 and plus_di.iloc[-1 - day_offset] > minus_di.iloc[-1 - day_offset])
        results['ADX'] = adx_score
        score += adx_score

        # Price Action & Trendlines
        pa_score = int(data['Close'].iloc[-1 - day_offset] > data['Close'].rolling(window=20).max().iloc[-2 - day_offset])
        results['Price Action'] = pa_score
        score += pa_score

        # Candlestick Patterns (Hammer)
        def hammer_pattern(data):
            open, close, low = data['Open'], data['Close'], data['Low']
            return ((close > open) and (open - low) > 2 * (close - open))

        hammer_score = int(hammer_pattern(data.iloc[-1 - day_offset]))
        results['Candlestick'] = hammer_score
        score += hammer_score

        # Ichimoku Cloud
        ichimoku = ta.trend.IchimokuIndicator(data['High'], data['Low'], window1=9, window2=26, window3=52)
        ich_score = int(data['Close'].iloc[-1 - day_offset] > ichimoku.ichimoku_a().iloc[-1 - day_offset] and data['Close'].iloc[-1 - day_offset] > ichimoku.ichimoku_b().iloc[-1 - day_offset])
        results['Ichimoku'] = ich_score
        score += ich_score

        return score, results

    # Get today's and yesterday's score
    score_today, results_today = get_score(data)
    score_yesterday, results_yesterday = get_score(data, day_offset=1)

    # Create result DataFrame
    result_df = pd.DataFrame([results_today])
    result_df['Stock'] = ticker
    result_df['Bullish Score Today'] = score_today
    result_df['Bullish Score Yesterday'] = score_yesterday
    result_df['Score Difference'] = score_today - score_yesterday

    # Add Today Change % and Yesterday Change %
    result_df['Today Change %'] = (data['Close'].iloc[-1] - data['Close'].iloc[-2]) / data['Close'].iloc[-2] * 100
    result_df['Yesterday Change %'] = (data['Close'].iloc[-2] - data['Close'].iloc[-3]) / data['Close'].iloc[-3] * 100

    # Reorder columns for better presentation
    columns_order = ['Stock'] + list(results_today.keys()) + ['Bullish Score Today', 'Bullish Score Yesterday', 'Score Difference', 'Today Change %', 'Yesterday Change %']
    result_df = result_df[columns_order]

    return result_df

# Main function to process multiple tickers
def main():
    # Read the Excel sheet with tickers
    global final_df2, final_df, result_df, all_results

    ticker_data = pd.read_excel(input_file_path, sheet_name=sheet_name)
    tickers = ticker_data['Ticker'].tolist()
    number_of_tickers = len(tickers)
    print("Number of tickers:", number_of_tickers)

    # Initialize a list to store results for each ticker
    all_results = []

    # Calculate indicators for each ticker and collect results
    for index, ticker in enumerate(tickers):
        print(f"Processing {index + 1}/{len(tickers)}: {ticker}...")
        result_df = calculate_indicators(ticker)
        if not result_df.empty:
            all_results.append(result_df)


    # Combine all results into a single DataFrame
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
        final_df = final_df[(final_df['Score Difference'] > 0) & (final_df['Bullish Score Today'] > 10) & (final_df['RSI'] == 1) & (final_df['MACD'] == 1) ]
        final_df = final_df.sort_values(by='Score Difference', ascending=False)
        final_df2 = final_df
        display(final_df2)

        # Save the results to the Excel file
        try:
            existing_df = pd.read_excel(output_file_path)
            final_df = pd.concat([existing_df, final_df], ignore_index=True)
        except FileNotFoundError:
            print(f"{output_file_path} not found. A new file will be created.")

        final_df.to_excel(output_file_path, index=False)
    else:
        print("No valid signals detected for any stock.")

# Execute the main function
if __name__ == "__main__":
    main()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=65dd47087c5deaad1105af8b9cf5d88f50be925c58fd25c30fb6a7fd81da0dff
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=6fc81d45244edd345f5296a83878afacc2faca37786a0bf742402f691dd530a4
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built pandas_ta ta
Mounted at /content/drive
Number of tickers: 1000
Processing 1/1000: AAPL...


[*********************100%***********************]  1 of 1 completed


ValueError: Data must be 1-dimensional, got ndarray of shape (252, 1) instead

In [ ]:
!pip install pandas_ta ta


# US Market: Few Public Traded Companies and its technical score as on Oct 29th 2024:


In [ ]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import ta
import numpy as np
from google.colab import drive
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module='ta.trend')

# Mount Google Drive to save files
drive.mount('/content/drive', force_remount=True)

# File paths
drive_path = '/content/drive/My Drive/Colab Notebooks'
output_file_path = f'{drive_path}/BB_Results_v5A_US1000_12cons_2RSI_Backtesting_v1.xlsx'

# Function to calculate indicators for a given ticker
def calculate_indicators(ticker):
    data = yf.download(ticker, period='1y', interval='1d')
    if data.empty or len(data) < 200:  # Ensure there is enough data for long-term indicators
        print(f"Insufficient data for {ticker}")
        return pd.DataFrame()  # Skip this stock

    # Calculate scores for today and yesterday
    def get_score(data, day_offset=0):
        score = 0
        results = {}

        # MACD
        macd = ta.trend.MACD(data['Close'])
        macd_score = int(macd.macd_diff().iloc[-1 - day_offset] > 0)
        results['MACD'] = macd_score
        score += macd_score

        # RSI with additional condition (RSI yesterday > RSI day before yesterday)
        rsi = ta.momentum.RSIIndicator(data['Close']).rsi()
        rsi_score = int(30 < rsi.iloc[-1 - day_offset] < 60 and rsi.iloc[-1 - day_offset] > rsi.iloc[-2 - day_offset])
        results['RSI'] = rsi_score
        score += rsi_score

        # Golden Cross
        sma50 = ta.trend.SMAIndicator(data['Close'], 50).sma_indicator()
        sma200 = ta.trend.SMAIndicator(data['Close'], 200).sma_indicator()
        ma_score = int(sma50.iloc[-1 - day_offset] > sma200.iloc[-1 - day_offset])
        results['Golden Cross'] = ma_score
        score += ma_score

        # Bollinger Bands
        bb = ta.volatility.BollingerBands(data['Close'])
        middle_band = bb.bollinger_mavg()  # This represents the 20-day SMA
        bb_score = int(data['Close'].iloc[-1 - day_offset] > middle_band.iloc[-1 - day_offset])
        results['Bollinger Bands'] = bb_score
        score += bb_score

        # Volume
        avg_vol = data['Volume'].rolling(window=20).mean()
        vol_score = int(data['Volume'].iloc[-1 - day_offset] > avg_vol.iloc[-1 - day_offset])
        results['Volume'] = vol_score
        score += vol_score

        # OBV
        obv = ta.volume.OnBalanceVolumeIndicator(data['Close'], data['Volume']).on_balance_volume()
        obv_score = int(obv.diff().iloc[-1 - day_offset] > 0)
        results['OBV'] = obv_score
        score += obv_score

        # Accumulation/Distribution Line
        ad = ta.volume.AccDistIndexIndicator(data['High'], data['Low'], data['Close'], data['Volume']).acc_dist_index()
        ad_score = int(ad.diff().iloc[-1 - day_offset] > 0)
        results['Acc/Dist Line'] = ad_score
        score += ad_score

        # Chaikin Money Flow
        cmf = ta.volume.ChaikinMoneyFlowIndicator(data['High'], data['Low'], data['Close'], data['Volume']).chaikin_money_flow()
        cmf_score = int(cmf.iloc[-1 - day_offset] > 0)
        results['CMF'] = cmf_score
        score += cmf_score

        # Parabolic SAR
        psar = ta.trend.PSARIndicator(data['High'], data['Low'], data['Close']).psar()
        psar_score = int(data['Close'].iloc[-1 - day_offset] > psar.iloc[-1 - day_offset])
        results['Parabolic SAR'] = psar_score
        score += psar_score

        # Stochastic Oscillator
        stoch = ta.momentum.StochasticOscillator(data['High'], data['Low'], data['Close'])
        stoch_score = int(stoch.stoch_signal().iloc[-1 - day_offset] > stoch.stoch().iloc[-1 - day_offset] and stoch.stoch().iloc[-1 - day_offset] < 20)
        results['Stochastic'] = stoch_score
        score += stoch_score

        # Fibonacci Retracement
        max_price = data['Close'].max()
        min_price = data['Close'].min()
        fib_50 = min_price + (max_price - min_price) * 0.5
        fib_score = int(data['Close'].iloc[-1 - day_offset] > fib_50)
        results['Fib Retracement'] = fib_score
        score += fib_score

        # ADX
        adx = ta.trend.ADXIndicator(data['High'], data['Low'], data['Close']).adx()
        plus_di = ta.trend.ADXIndicator(data['High'], data['Low'], data['Close']).adx_pos()
        minus_di = ta.trend.ADXIndicator(data['High'], data['Low'], data['Close']).adx_neg()
        adx_score = int(adx.iloc[-1 - day_offset] > 20 and plus_di.iloc[-1 - day_offset] > minus_di.iloc[-1 - day_offset])
        results['ADX'] = adx_score
        score += adx_score

        # Price Action & Trendlines
        pa_score = int(data['Close'].iloc[-1 - day_offset] > data['Close'].rolling(window=20).max().iloc[-2 - day_offset])
        results['Price Action'] = pa_score
        score += pa_score

        # Candlestick Patterns (Hammer)
        def hammer_pattern(data):
            open, close, low = data['Open'], data['Close'], data['Low']
            return ((close > open) and (open - low) > 2 * (close - open))

        hammer_score = int(hammer_pattern(data.iloc[-1 - day_offset]))
        results['Candlestick'] = hammer_score
        score += hammer_score

        # Ichimoku Cloud
        ichimoku = ta.trend.IchimokuIndicator(data['High'], data['Low'], window1=9, window2=26, window3=52)
        ich_score = int(data['Close'].iloc[-1 - day_offset] > ichimoku.ichimoku_a().iloc[-1 - day_offset] and data['Close'].iloc[-1 - day_offset] > ichimoku.ichimoku_b().iloc[-1 - day_offset])
        results['Ichimoku'] = ich_score
        score += ich_score

        return score, results

    # Get today's and yesterday's score
    score_today, results_today = get_score(data)
    score_yesterday, results_yesterday = get_score(data, day_offset=1)

    # Create result DataFrame
    result_df = pd.DataFrame([results_today])
    result_df['Stock'] = ticker
    result_df['Bullish Score Today'] = score_today
    result_df['Bullish Score Yesterday'] = score_yesterday
    result_df['Score Difference'] = score_today - score_yesterday

    # Add Today Change % and Yesterday Change %
    result_df['Today Change %'] = (data['Close'].iloc[-1] - data['Close'].iloc[-2]) / data['Close'].iloc[-2] * 100
    result_df['Yesterday Change %'] = (data['Close'].iloc[-2] - data['Close'].iloc[-3]) / data['Close'].iloc[-3] * 100

    # Reorder columns for better presentation
    columns_order = ['Stock'] + list(results_today.keys()) + ['Bullish Score Today', 'Bullish Score Yesterday', 'Score Difference', 'Today Change %', 'Yesterday Change %']
    result_df = result_df[columns_order]

    return result_df

# Main function to process specific tickers
def main():
    # Hardcoded list of tickers you want to analyze
    tickers = ['AAPL', 'GOOGL', 'NSRGY', 'GEHC', 'FRSH', 'RHHBY', 'NVS', 'JNJ', 'AMZN', 'MSFT', 'NVDA', 'TSLA']
    number_of_tickers = len(tickers)
    print("Number of tickers:", number_of_tickers)

    # Initialize a list to store results for each ticker
    all_results = []

    # Calculate indicators for each ticker and collect results
    for index, ticker in enumerate(tickers):
        print(f"Processing {index + 1}/{len(tickers)}: {ticker}...")
        result_df = calculate_indicators(ticker)
        if not result_df.empty:
            all_results.append(result_df)

    # Combine all results into a single DataFrame
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
        #final_df = final_df[(final_df['Score Difference'] > 0) & (final_df['Bullish Score Today'] > 10) & (final_df['RSI'] == 1) & (final_df['MACD'] == 1) ]
        final_df = final_df.sort_values(by='Score Difference', ascending=False)
        final_df2 = final_df
        display(final_df2)

        # Save the results to the Excel file
        try:
            existing_df = pd.read_excel(output_file_path)
            final_df = pd.concat([existing_df, final_df], ignore_index=True)
        except FileNotFoundError:
            print(f"{output_file_path} not found. A new file will be created.")

        final_df.to_excel(output_file_path, index=False)
    else:
        print("No valid signals detected for any stock.")

# Execute the main function
if __name__ == "__main__":
    main()


# Swiss Market: Top 30 Companies

In [ ]:
print("• Nestlé (NESN.SW) - Market Cap: $340 billion USD, Revenue: $95 billion USD, Employees: 270,000, PE Ratio: 25, Key Products: Food & Beverages\n")
print("• Roche Holding (ROG.SW) - Market Cap: $250 billion USD, Revenue: $70 billion USD, Employees: 100,000, PE Ratio: 21, Key Products: Pharmaceuticals & Diagnostics\n")
print("• Novartis (NOVN.SW) - Market Cap: $200 billion USD, Revenue: $53 billion USD, Employees: 100,000, PE Ratio: 20, Key Products: Pharmaceuticals & Gene Therapies\n")
print("• Zurich Insurance Group (ZURN.SW) - Market Cap: $70 billion USD, Revenue: $50 billion USD, Employees: 56,000, PE Ratio: 12, Key Products: Insurance & Asset Management\n")
print("• UBS Group (UBSG.SW) - Market Cap: $60 billion USD, Revenue: $35 billion USD, Employees: 72,000, PE Ratio: 9, Key Products: Banking & Wealth Management\n")
print("• ABB Ltd (ABBN.SW) - Market Cap: $60 billion USD, Revenue: $30 billion USD, Employees: 105,000, PE Ratio: 18, Key Products: Robotics & Automation\n")
print("• Cie Financière Richemont (CFR.SW) - Market Cap: $60 billion USD, Revenue: $20 billion USD, Employees: 35,000, PE Ratio: 32, Key Products: Luxury Goods & Jewelry\n")
print("• Alcon (ALC.SW) - Market Cap: $40 billion USD, Revenue: $8 billion USD, Employees: 24,000, PE Ratio: 40, Key Products: Eye Care & Surgical Products\n")
print("• Givaudan (GIVN.SW) - Market Cap: $40 billion USD, Revenue: $7 billion USD, Employees: 16,000, PE Ratio: 35, Key Products: Flavors & Fragrances\n")
print("• Swiss Re (SREN.SW) - Market Cap: $30 billion USD, Revenue: $40 billion USD, Employees: 14,000, PE Ratio: 11, Key Products: Reinsurance\n")
print("• Lonza Group (LONN.SW) - Market Cap: $50 billion USD, Revenue: $6 billion USD, Employees: 17,000, PE Ratio: 25, Key Products: Biotech Solutions & Manufacturing\n")
print("• Swisscom (SCMN.SW) - Market Cap: $30 billion USD, Revenue: $12 billion USD, Employees: 19,000, PE Ratio: 16, Key Products: Telecommunications\n")
print("• Partners Group (PGHN.SW) - Market Cap: $25 billion USD, Revenue: $2 billion USD, Employees: 1,800, PE Ratio: 22, Key Products: Private Equity & Asset Management\n")
print("• Geberit (GEBN.SW) - Market Cap: $20 billion USD, Revenue: $4 billion USD, Employees: 12,000, PE Ratio: 30, Key Products: Sanitary Products\n")
print("• Sika (SIKA.SW) - Market Cap: $40 billion USD, Revenue: $10 billion USD, Employees: 25,000, PE Ratio: 33, Key Products: Construction Chemicals\n")
print("• Julius Baer Group (BAER.SW) - Market Cap: $15 billion USD, Revenue: $4 billion USD, Employees: 6,700, PE Ratio: 13, Key Products: Private Banking\n")
print("• Schindler Holding (SCHN.SW) - Market Cap: $20 billion USD, Revenue: $12 billion USD, Employees: 69,000, PE Ratio: 18, Key Products: Elevators & Escalators\n")
print("• SGS (SGSN.SW) - Market Cap: $20 billion USD, Revenue: $7 billion USD, Employees: 97,000, PE Ratio: 28, Key Products: Inspection & Certification\n")
print("• Logitech International (LOGN.SW) - Market Cap: $10 billion USD, Revenue: $5 billion USD, Employees: 7,000, PE Ratio: 17, Key Products: Computer Accessories\n")
print("• Credit Suisse (CSGN.SW) - Market Cap: $10 billion USD, Revenue: $20 billion USD, Employees: 50,000, PE Ratio: N/A (recent restructuring), Key Products: Banking & Investment\n")
print("• Adecco Group (ADEN.SW) - Market Cap: $8 billion USD, Revenue: $25 billion USD, Employees: 30,000, PE Ratio: 11, Key Products: Staffing & HR Solutions\n")
print("• Lindt & Sprüngli (LISN.SW) - Market Cap: $25 billion USD, Revenue: $5 billion USD, Employees: 14,000, PE Ratio: 45, Key Products: Chocolate & Confectionery\n")
print("• Sonova Holding (SOON.SW) - Market Cap: $20 billion USD, Revenue: $4 billion USD, Employees: 17,000, PE Ratio: 26, Key Products: Hearing Aids\n")
print("• Straumann Holding (STMN.SW) - Market Cap: $20 billion USD, Revenue: $2 billion USD, Employees: 9,000, PE Ratio: 35, Key Products: Dental Implants\n")
print("• Temenos (TEMN.SW) - Market Cap: $8 billion USD, Revenue: $1 billion USD, Employees: 4,000, PE Ratio: 20, Key Products: Banking Software\n")
print("• Helvetia Holding (HELN.SW) - Market Cap: $5 billion USD, Revenue: $10 billion USD, Employees: 11,000, PE Ratio: 8, Key Products: Insurance\n")
print("• Baloise Holding (BALN.SW) - Market Cap: $7 billion USD, Revenue: $9 billion USD, Employees: 7,500, PE Ratio: 9, Key Products: Insurance & Pensions\n")
print("• Clariant (CLN.SW) - Market Cap: $5 billion USD, Revenue: $4 billion USD, Employees: 13,000, PE Ratio: 18, Key Products: Specialty Chemicals\n")
#print("• Kuehne + Nagel (KNIN.SW) - Market Cap: $30 billion USD, Revenue: $30 billion USD, Employees: 79,000, PE Ratio: 15, Key Products: Logistics & Freight\n")
#print("• Dufry (DUFN.SW) - Market Cap: $5 billion USD, Revenue: $7 billion USD, Employees: 20,000, PE Ratio: 12, Key Products: Travel Retail\n")


• Nestlé (NESN.SW) - Market Cap: $340 billion USD, Revenue: $95 billion USD, Employees: 270,000, PE Ratio: 25, Key Products: Food & Beverages

• Roche Holding (ROG.SW) - Market Cap: $250 billion USD, Revenue: $70 billion USD, Employees: 100,000, PE Ratio: 21, Key Products: Pharmaceuticals & Diagnostics

• Novartis (NOVN.SW) - Market Cap: $200 billion USD, Revenue: $53 billion USD, Employees: 100,000, PE Ratio: 20, Key Products: Pharmaceuticals & Gene Therapies

• Zurich Insurance Group (ZURN.SW) - Market Cap: $70 billion USD, Revenue: $50 billion USD, Employees: 56,000, PE Ratio: 12, Key Products: Insurance & Asset Management

• UBS Group (UBSG.SW) - Market Cap: $60 billion USD, Revenue: $35 billion USD, Employees: 72,000, PE Ratio: 9, Key Products: Banking & Wealth Management

• ABB Ltd (ABBN.SW) - Market Cap: $60 billion USD, Revenue: $30 billion USD, Employees: 105,000, PE Ratio: 18, Key Products: Robotics & Automation

• Cie Financière Richemont (CFR.SW) - Market Cap: $60 billion U

In [ ]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import ta
import numpy as np
from google.colab import drive
import warnings

warnings.filterwarnings("ignore", category=FutureWarning, module='ta.trend')

# Mount Google Drive to save files
drive.mount('/content/drive', force_remount=True)

# File paths
drive_path = '/content/drive/My Drive/Colab Notebooks'
output_file_path = f'{drive_path}/BB_Results_v5A_US1000_12cons_2RSI_Backtesting_v1.xlsx'

# Function to calculate indicators for a given ticker
def calculate_indicators(ticker):
    data = yf.download(ticker, period='1y', interval='1d')
    if data.empty or len(data) < 200:  # Ensure there is enough data for long-term indicators
        print(f"Insufficient data for {ticker}")
        return pd.DataFrame()  # Skip this stock

    # Calculate scores for today and yesterday
    def get_score(data, day_offset=0):
        score = 0
        results = {}

        # MACD
        macd = ta.trend.MACD(data['Close']).macd_diff()
        macd_score = int(macd.iloc[-1 - day_offset] > 0)
        results['MACD'] = macd_score
        score += macd_score

        # RSI with additional condition (RSI yesterday > RSI day before yesterday)
        rsi = ta.momentum.RSIIndicator(data['Close']).rsi()
        rsi_score = int(30 < rsi.iloc[-1 - day_offset] < 60 and rsi.iloc[-1 - day_offset] > rsi.iloc[-2 - day_offset])
        results['RSI'] = rsi_score
        score += rsi_score

        # Golden Cross
        sma50 = ta.trend.SMAIndicator(data['Close'], 50).sma_indicator()
        sma200 = ta.trend.SMAIndicator(data['Close'], 200).sma_indicator()
        ma_score = int(sma50.iloc[-1 - day_offset] > sma200.iloc[-1 - day_offset])
        results['Golden Cross'] = ma_score
        score += ma_score

        # Bollinger Bands
        bb = ta.volatility.BollingerBands(data['Close'])
        middle_band = bb.bollinger_mavg()
        bb_score = int(data['Close'].iloc[-1 - day_offset] > middle_band.iloc[-1 - day_offset])
        results['Bollinger Bands'] = bb_score
        score += bb_score

        # Volume
        avg_vol = data['Volume'].rolling(window=20).mean()
        vol_score = int(data['Volume'].iloc[-1 - day_offset] > avg_vol.iloc[-1 - day_offset])
        results['Volume'] = vol_score
        score += vol_score

        # OBV
        obv = ta.volume.OnBalanceVolumeIndicator(data['Close'], data['Volume']).on_balance_volume()
        obv_score = int(obv.diff().iloc[-1 - day_offset] > 0)
        results['OBV'] = obv_score
        score += obv_score

        # Accumulation/Distribution Line
        ad = ta.volume.AccDistIndexIndicator(data['High'], data['Low'], data['Close'], data['Volume']).acc_dist_index()
        ad_score = int(ad.diff().iloc[-1 - day_offset] > 0)
        results['Acc/Dist Line'] = ad_score
        score += ad_score

        # Chaikin Money Flow
        cmf = ta.volume.ChaikinMoneyFlowIndicator(data['High'], data['Low'], data['Close'], data['Volume']).chaikin_money_flow()
        cmf_score = int(cmf.iloc[-1 - day_offset] > 0)
        results['CMF'] = cmf_score
        score += cmf_score

        # Parabolic SAR
        psar = ta.trend.PSARIndicator(data['High'], data['Low'], data['Close']).psar()
        psar_score = int(data['Close'].iloc[-1 - day_offset] > psar.iloc[-1 - day_offset])
        results['Parabolic SAR'] = psar_score
        score += psar_score

        # Stochastic Oscillator
        stoch = ta.momentum.StochasticOscillator(data['High'], data['Low'], data['Close'])
        stoch_score = int(stoch.stoch_signal().iloc[-1 - day_offset] > stoch.stoch().iloc[-1 - day_offset] and stoch.stoch().iloc[-1 - day_offset] < 20)
        results['Stochastic'] = stoch_score
        score += stoch_score

        # Fibonacci Retracement
        max_price = data['Close'].max()
        min_price = data['Close'].min()
        fib_50 = min_price + (max_price - min_price) * 0.5
        fib_score = int(data['Close'].iloc[-1 - day_offset] > fib_50)
        results['Fib Retracement'] = fib_score
        score += fib_score

        # ADX
        adx = ta.trend.ADXIndicator(data['High'], data['Low'], data['Close']).adx()
        plus_di = ta.trend.ADXIndicator(data['High'], data['Low'], data['Close']).adx_pos()
        minus_di = ta.trend.ADXIndicator(data['High'], data['Low'], data['Close']).adx_neg()
        adx_score = int(adx.iloc[-1 - day_offset] > 20 and plus_di.iloc[-1 - day_offset] > minus_di.iloc[-1 - day_offset])
        results['ADX'] = adx_score
        score += adx_score

        # Price Action & Trendlines
        pa_score = int(data['Close'].iloc[-1 - day_offset] > data['Close'].rolling(window=20).max().iloc[-2 - day_offset])
        results['Price Action'] = pa_score
        score += pa_score

        # Candlestick Patterns (Hammer)
        def hammer_pattern(data):
            open, close, low = data['Open'], data['Close'], data['Low']
            return ((close > open) and (open - low) > 2 * (close - open))

        hammer_score = int(hammer_pattern(data.iloc[-1 - day_offset]))
        results['Candlestick'] = hammer_score
        score += hammer_score

        # Ichimoku Cloud
        ichimoku = ta.trend.IchimokuIndicator(data['High'], data['Low'], window1=9, window2=26, window3=52)
        ich_score = int(data['Close'].iloc[-1 - day_offset] > ichimoku.ichimoku_a().iloc[-1 - day_offset] and data['Close'].iloc[-1 - day_offset] > ichimoku.ichimoku_b().iloc[-1 - day_offset])
        results['Ichimoku'] = ich_score
        score += ich_score

        return score, results

    # Get today's and yesterday's score
    score_today, results_today = get_score(data)
    score_yesterday, results_yesterday = get_score(data, day_offset=1)

    # Create result DataFrame
    result_df = pd.DataFrame([results_today])
    result_df['Stock'] = ticker
    result_df['Bullish Score Today'] = score_today
    result_df['Bullish Score Yesterday'] = score_yesterday
    result_df['Score Difference'] = score_today - score_yesterday

    # Add Today Change % and Yesterday Change %
    result_df['Today Change %'] = (data['Close'].iloc[-1] - data['Close'].iloc[-2]) / data['Close'].iloc[-2] * 100
    result_df['Yesterday Change %'] = (data['Close'].iloc[-2] - data['Close'].iloc[-3]) / data['Close'].iloc[-3] * 100

    # Reorder columns for better presentation
    columns_order = ['Stock'] + list(results_today.keys()) + ['Bullish Score Today', 'Bullish Score Yesterday', 'Score Difference', 'Today Change %', 'Yesterday Change %']
    result_df = result_df[columns_order]

    return result_df

# Main function to process specific tickers
def main():
    # Hardcoded list of tickers you want to analyze
    tickers = [
        'NESN.SW', 'ROG.SW', 'NOVN.SW', 'ZURN.SW', 'UBSG.SW', 'ABBN.SW',
        'CFR.SW', 'ALC.SW', 'GIVN.SW', 'SREN.SW', 'LONN.SW', 'SCMN.SW',
        'PGHN.SW', 'GEBN.SW', 'SIKA.SW', 'BAER.SW', 'SCHN.SW', 'SGSN.SW',
        'LOGN.SW', 'CSGN.SW', 'ADEN.SW', 'LISN.SW', 'SOON.SW', 'STMN.SW',
        'TEMN.SW', 'HELN.SW', 'BALN.SW', 'CLN.SW', 'KNIN.SW', 'DUFN.SW'
    ]
    number_of_tickers = len(tickers)
    print("Number of tickers:", number_of_tickers)

    # Initialize a list to store results for each ticker
    all_results = []

    # Calculate indicators for each ticker and collect results
    for index, ticker in enumerate(tickers):
        print(f"Processing {index + 1}/{len(tickers)}: {ticker}...")
        result_df = calculate_indicators(ticker)
        if not result_df.empty:
            all_results.append(result_df)

    # Combine all results into a single DataFrame
    if all_results:
        final_df = pd.concat(all_results, ignore_index=True)
        final_df = final_df.sort_values(by='Score Difference', ascending=False)
        final_df2 = final_df
        display(final_df2)

        # Save the results to the Excel file
        try:
            existing_df = pd.read_excel(output_file_path)
            final_df = pd.concat([existing_df, final_df], ignore_index=True)
        except FileNotFoundError:
            print(f"{output_file_path} not found. A new file will be created.")

        final_df.to_excel(output_file_path, index=False)
    else:
        print("No valid signals detected for any stock.")

# Execute the main function
if __name__ == "__main__":
    main()


Mounted at /content/drive
Number of tickers: 30
Processing 1/30: NESN.SW...


[*********************100%***********************]  1 of 1 completed


ValueError: Data must be 1-dimensional, got ndarray of shape (252, 1) instead

In [ ]:
# Note: Financial markets are subject to highly volatile and non-stationary. AI and Data exploration purpose.